**Q1:** 

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


#------------ READING THE .html ---------------------------
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]


mask = cities["NHL"] == "—"                        # first mask dropping the columns with "-" as value
cities = cities[~mask]                             # applying the first mask
cities = cities.sort_values("Metropolitan area")   # sorting values, by alphabetical, to fix the index
cities = cities.reset_index(drop=True)             # reseting the index


#--------------- CLEANING SOME VALUES ---------
vals1=[] 
for item in cities["NHL"]:
    item = re.sub("\[\w*\s\w*\]","", item)         # using regex to replace '[note X]' for ""
    vals1.append(item)
cities["NHL"] = vals1                              # applying changes


mask2 = cities["NHL"] == ""                        # second mask to drop the empy values ' "" ' previously defined
cities = cities[~mask2]

cities = cities.sort_values('Metropolitan area')   # sorting values again to fix the index
cities = cities.reset_index(drop=True)             # fixing the index




#------------------------ READING THE CSV  ---------------
nhl_df=pd.read_csv("assets/nhl.csv") 

#---------- SELECTING ONLY 2018' DATA FROM CSV FILE ---------
mask_nhl = nhl_df["year"] == 2018
nhl_df = nhl_df[mask_nhl]
nhl_df = nhl_df[["team","W", "L", "year"]]


nhl_df = nhl_df.drop([0,9,18,26])           #dropping "[...] Division" columns
nhl_df = nhl_df.reset_index(drop=True)




#------- CLEANING THE '*' FROM TEAM COLUMN -------
vals=[] 
for item in nhl_df["team"]:
    item = re.sub("(\*{1})","", item) 
    vals.append(item)
nhl_df["team"] = vals    
    
    
    

#----------- SPLITING THE TEAM COLUNM IN CITY NAME AND ONLY TEAM NAME -------------
nm = []
ct = []
sel = ["Tampa Bay Lightning","New Jersey Devils", "New York Islanders",
       "New York Rangers", "St. Louis Blues", "San Jose Sharks", 
       "Los Angeles Kings"] # list helping to fix the problem with 2-word cities 

for item in nhl_df["team"]:
    x = item.split()        # spliting every word, x will be a list
    nm.append(x[-1])        # appending the team name to 'nm' list 
    if item in sel:         # solving the problem with 2-word cities 
        ct.append(x[0:2])   # appending the 2 first itens in x list with city name in 'sel' list
    else:
        ct.append(x[0])
    
    

nhl_df['names'] = nm  #column for team names

dic_nm ={"Leafs": "Maple Leafs", "Wings":"Red Wings", "Jackets":"Blue Jackets",
         "Knights":"Golden Knights"}                                            # fixing the problem with 2-word team names 
nhl_df['names'] = nhl_df.names.map(dic_nm).fillna(nhl_df['names'])              # the fillna() function tells map to keep 
                                                                                # old values if not in replacement dict







cities_nm =[]                                        # just another list for city names
for item in ct:                                      # fixing the problem with city names list, 2-word city names are 
    if type(item) == list:                           # a list inside other list, try 'print(ct)'
        l = " ".join(str(g) for g in item)           # using '.join()' only for lists in 'ct' list
        cities_nm.append(l)
    else:
        cities_nm.append(item)
        
nhl_df['team'] = cities_nm                           #applying the list to a column                                   
nhl_df = nhl_df.sort_values('team', ascending=True)  # sorting by alphabetical order on 'team' column
nhl_df.set_index('team', inplace=True)               #setting index for better visualization




#--------- TRANSFORMING ENTRIES FROM STR TO FLOAT ------------------

nhl_df["W"] = nhl_df["W"].astype('float')
nhl_df["L"] = nhl_df["L"].astype('float')


cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype('float')


#-------------- RATIO W/L CALCULATION  -------------
nhl_df["W/L"] = nhl_df["W"]/(nhl_df["W"]+nhl_df["L"])





#----------FIXING NEW YORKS PROBLEM--------------------
ny_df = nhl_df.loc[["New York", "New Jersey"]]
Ny = pd.DataFrame([[sum(ny_df["W"]), sum(ny_df["L"]), 2018, "RangersIslandersDevils",
                   np.mean(ny_df["W/L"])]], columns=['W', 'L', 'year', 'names', 'W/L'], index=["New York City"])

nhl_df = pd.concat([nhl_df, Ny])


#----------FIXING LOS ANGELES' PROBLEM --------------------
la_df = nhl_df.loc[["Los Angeles", "Anaheim"]]
La = pd.DataFrame([[sum(la_df["W"]), sum(la_df["L"]), 2018, "KingsDucks",
                   np.mean(la_df["W/L"])]], columns=['W', 'L', 'year', 'names', 'W/L'], index=["Los Angeles"])

nhl_df = pd.concat([nhl_df, La])


#------------ DELETING SAME REGIONS TEAMS  --------------------
del_names = ["Ducks", "Kings", "Rangers", "Islanders", "Devils"]

nhl_df = nhl_df[~nhl_df["names"].isin(del_names)]
nhl_df=nhl_df.reset_index(drop=False) #FIXING INDEX
nhl_df.columns=["cities", 'W', 'L', 'year', 'names', 'W/L']



#------------ MERGING THE DATAFRAMES BY TEAM'S NAMES --------------------
final_df = pd.merge(cities, nhl_df, how='inner', left_on='NHL', right_on="names")
final_df = final_df[["Metropolitan area","Population (2016 est.)[8]", "NHL", "W", "L", "W/L"]]
final_df




def nhl_correlation():

    DF = final_df.copy()
    population_by_region = DF["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = DF["W/L"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]




nhl_correlation()
                          


#final_df

####**Q2:**
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba=pd.read_csv("assets/nba.csv")
CT=pd.read_html("assets/wikipedia_data.html")[1]
CT=CT.iloc[:-1,[0,3,5,6,7,8]]


#nba = nba_df.copy()
nba = nba[["team", "W", "L", "year"]]
mask1_nba = nba["year"] == 2018
nba = nba[mask1_nba]


#------- CLEANING THE '*' AND '(X)' FROM TEAM COLUMN -------
vals1=[]
vals2=[]
vals3=[]
for item in nba['team']:
    item = re.sub('(\(\w*\))',"", item)
    vals1.append(item)
nba['team'] = vals1
for item in nba["team"]:
    item = re.sub("(\*{1})","", item) 
    vals2.append(item)
nba["team"] = vals2

for item in nba["team"]:
    item = re.sub(r"[\n\t]*", "", item)
    vals3.append(item)
    
nba["team"] = vals3



two_word_cities = ["New York Knicks", "Los Angeles Clippers", "San Antonio Spurs", "Los Angeles Lakers"]
nm    = []
citis = []
for item in nba["team"]:
    x = item.split()
    nm.append(x[-1])
    citis.append(x[0])
    
        
        
    
    
nba["names"] = nm
nba["cities"] = citis
dic_nm = {"Blazers":"Trail Blazers"}
nba["names"] = nba.names.map(dic_nm).fillna(nba["names"])
dic_ct={"New":"New York City", "Golden":"San Francisco Bay", "San":"San Antonio"
,"Los":"Los Angeles", "Oklahoma":"Oklahoma City", "Miami":"Miami–Fort Lauderdale",
       "Utah":"Salt Lake City", "Washington":"Washington, D.C.","Brooklyn":"New York City"}

nba["cities"] = nba.cities.map(dic_ct).fillna(nba["cities"])




#--------- TRANSFORMING ENTRIES FROM STR TO FLOAT ------------------

nba["W"] = nba["W"].astype('float')
nba["L"] = nba["L"].astype('float')


CT['Population (2016 est.)[8]'] = CT['Population (2016 est.)[8]'].astype('float')


#-------------- RATIO W/L CALCULATION  -------------
nba["W/L"] = nba["W"]/(nba["W"]+nba["L"])


nba["team"] = nba["cities"]

nba = nba[["team", "W", "L", "names","W/L"]]
nba.sort_values('team', inplace=True)

#----------FIXING NEW ORLEANS PROBLEM--------------------
no_df = nba.loc[nba['names']=="Pelicans"]
No = pd.DataFrame([["New Orleans",sum(no_df["W"]), sum(no_df["L"]), "Pelicans", sum(no_df["W/L"])]],
                  columns=['team',"W", "L", "names", "W/L"])

nba = pd.concat([nba, No])


nba.sort_values('team', inplace=True)
nba.reset_index(drop=True, inplace=True)
nba = nba.drop(18)

#----------FIXING NEW YORKS PROBLEM--------------------
ny_df = nba.loc[nba['team']=="New York City"]
ny_df
Ny = pd.DataFrame([["New York City", sum(ny_df["W"]), sum(ny_df["L"]), "KnicksNets",
                   np.mean(ny_df["W/L"])]], columns=['team','W', 'L', 'names', 'W/L'])

nba = pd.concat([nba, Ny])


#----------FIXING LOS ANGELES' PROBLEM --------------------
la_df = nba.loc[nba["team"]=="Los Angeles"]
#la_df
La = pd.DataFrame([["Los Angeles",sum(la_df["W"]), sum(la_df["L"]),"LakersClippers",
                   np.mean(la_df["W/L"])]], columns=['team','W', 'L','names', 'W/L'])
nba = pd.concat([nba, La])


#------------ DELETING SAME REGIONS TEAMS  --------------------
del_names = ["Clippers", "Lakers", "Knicks", "Nets"]

nba = nba[~nba["names"].isin(del_names)]
nba = nba.reset_index(drop=True) #FIXING INDEX
nba = nba.sort_values('team')
nba.columns=["team", 'W', 'L','names', 'W/L']



#------------ POPULATION AND REGION DATAFRAME ------------------------

CT = CT[['Metropolitan area', 'Population (2016 est.)[8]','NBA']]

mask1_CT = CT["NBA"] == '—'
CT = CT[~mask1_CT]
vals2=[]
for item in CT["NBA"]:
    item = re.sub("(\[\w*\s\w*\])","", item) 
    vals2.append(item)
    
CT["NBA"] = vals2

mask2_CT = CT["NBA"] == ""
CT = CT[~mask2_CT]
#----------- STR TO FLOAT -----------------
CT["Population (2016 est.)[8]"] = CT["Population (2016 est.)[8]"].astype('int64')

CT.sort_values('Metropolitan area', inplace=True)
CT.reset_index(drop=True, inplace=True)








final_df2 = pd.merge(CT, nba, how='inner', left_on='NBA', right_on="names")
final_df2 = final_df2[["Metropolitan area","Population (2016 est.)[8]", "NBA", "W", "L", "W/L"]]
final_df2.sort_values("Metropolitan area", inplace=True)
#final_df2





def nba_correlation():

    DF = final_df2.copy()
    population_by_region = DF["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = DF["W/L"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]




nba_correlation()




## **Q3:**##
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

mlb = mlb_df[["team", "W", "L", "year"]].copy()
CT = cities[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]].copy()


#----------- CLEANING CT DATAFRAME ----------------------
vals=[]
for item in CT["MLB"]:
    item = re.sub("(\[\w*\s\w*\])","", item) 
    vals.append(item)
CT["MLB"]=vals

mask_ct = CT["MLB"].isin(["—", ""])
CT = CT[~mask_ct]
CT.reset_index(drop=True, inplace=True)



#----------- CLEANING MLB DATAFRAME --------------
mask_mlb = mlb["year"]==2018
mlb = mlb[mask_mlb]
vals2=[]
for item in mlb["team"]:
    item = re.sub(r"[\n\t*]*", "", item)
    vals2.append(item)
    
mlb["team"] = vals2


two_word_cities = ["New York Yankees", "Tampa Bay Rays", "Kansas City Royals","Los Angeles Angels",
                   "New York Mets", "St. Louis Cardinals", "Los Angeles Dodgers","San Francisco Giants", "San Diego Padres"]
two_word_teams = ["Toronto Blue Jays", "Chicago White Sox","Boston Red Sox" ]



nm = []
ct_nm = []

for item in mlb['team']:
    x = item.split()
    if item in two_word_teams:
        nm.append(x[-2:])
    else:
        nm.append(x[-1])
    if item in two_word_cities:
        ct_nm.append(x[0:2])
    else:
        ct_nm.append(x[0])
        
        
ct =[]        
for item in ct_nm:
    if type(item)==list:
        l = " ".join(str(g) for g in item)
        ct.append(l)
    else:
        ct.append(item)
        
n_ms=[]
for item in nm:
    if type(item)==list:
        l = " ".join(str(g) for g in item)
        n_ms.append(l)
    else:
        n_ms.append(item)
        

mlb["MetroRegion"]= ct
mlb["names"] = n_ms
mlb=mlb[["MetroRegion", "names", "W", "L"]]


#------------- STR TO FLOAT ENTRIES ----------------

mlb["W"] = mlb["W"].astype('float')
mlb["L"] = mlb["L"].astype('float')


CT['Population (2016 est.)[8]'] = CT['Population (2016 est.)[8]'].astype('float')


#-------------- RATIO W/L CALCULATION  -------------
mlb["W/L"] = mlb["W"]/(mlb["W"]+mlb["L"])



#----------FIXING NEW YORKS PROBLEM--------------------
ny_df = mlb.loc[mlb['MetroRegion']=="New York"]
Ny = pd.DataFrame([["New York City","YankeesMets", sum(ny_df["W"]), sum(ny_df["L"]),
                   np.mean(ny_df["W/L"])]], columns=['MetroRegion', 'names','W', 'L', 'W/L'])

mlb = pd.concat([mlb, Ny])


#----------FIXING LOS ANGELES PROBLEM--------------------
la_df = mlb.loc[mlb['MetroRegion']=="Los Angeles"]
La = pd.DataFrame([["Los Angeles","DodgersAngels", sum(la_df["W"]), sum(la_df["L"]),
                   np.mean(la_df["W/L"])]], columns=['MetroRegion', 'names','W', 'L', 'W/L'])

mlb = pd.concat([mlb, La])


#----------FIXING CHICAGOS PROBLEM--------------------
ch_df = mlb.loc[mlb['MetroRegion']=="Chicago"]
Ch = pd.DataFrame([["Chicago","CubsWhite Sox", sum(ch_df["W"]), sum(ch_df["L"]),
                   np.mean(ch_df["W/L"])]], columns=['MetroRegion', 'names','W', 'L', 'W/L'])

mlb = pd.concat([mlb, Ch])

#------- FIXING SAN FRAN BAYS PROBLEM ---------------
sa_df = mlb.loc[mlb['names'].isin(["Giants", "Athletics"])]
Sa = pd.DataFrame([["San Francisco Bay Area","GiantsAthletics", sum(sa_df["W"]), sum(sa_df["L"]),
                   np.mean(sa_df["W/L"])]], columns=['MetroRegion', 'names','W', 'L', 'W/L'])

mlb = pd.concat([mlb, Sa])


#------------ DELETING SAME REGIONS TEAMS  --------------------
del_names = ["Angels", "Dodgers", "Giants", "Athletics", "Cubs", "White Sox", "Yankees", "Mets"]

mlb = mlb[~mlb["names"].isin(del_names)]
mlb.sort_values('MetroRegion', inplace=True)
mlb=mlb.reset_index(drop=True) #FIXING INDEX

'''
DodgersAngels ---- Los Angeles
GiantsAthletics ---- San Francisco Bay Area
CubsWhite Sox ---- Chicago
'''



#--------- FIXING SOME REGIONS DETAILS -------------------------
dic_nm={"Tampa Bay":"Tampa Bay Area", "Arizona":"Phoenix", "Miami":"Miami–Fort Lauderdale",
       "Minnesota":"Minneapolis–Saint Paul", "Texas":"Dallas–Fort Worth", "Washington":"Washington, D.C."}

mlb["MetroRegion"] == mlb.names.map(dic_nm).fillna(mlb["MetroRegion"])

#--------- MERGING BOTH DATAFRAMES ---------

final_df3 = pd.merge(mlb, CT, how='inner', left_on="names", right_on="MLB")
final_df3 = final_df3[["Metropolitan area", "Population (2016 est.)[8]", "W", "L", "W/L", "MLB" ]]

final_df




def mlb_correlation(): 
    DF = final_df3.copy()
    population_by_region = DF["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = DF["W/L"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]



mlb_correlation()



##**Q4:**

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
## import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


nfl = nfl_df[["team", "year", "W", "L"]].copy()
CT = cities[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]].copy()




mask_nfl= nfl["year"]==2018

nfl=nfl[mask_nfl]
nfl.drop([0,5,10,15,20,25,30,35], inplace=True)



#----------- CLEANING CT DATAFRAME ----------------------
vals2=[]
for item in CT["NFL"]:
    item = re.sub(r"[\n\t*]*", "", item)
    vals2.append(item)
    
CT["NFL"] = vals2
vals=[]
for item in CT["NFL"]:
    item = re.sub("(\[\w*\s\w*\])","", item) 
    vals.append(item)
CT["NFL"]=vals

mask_ct = CT["NFL"].isin(["—", "", "—", "—-"])
CT = CT[~mask_ct]
CT.reset_index(drop=True, inplace=True)



CT = CT.drop(13)
CT.reset_index(drop=True, inplace=True)



nfl
vals3=[]
for item in nfl["team"]:
    item = re.sub(r'(\**)(\+*)', "", item)
    vals3.append(item)
    
    
nfl["team"] = vals3
    
    
    
    
two_word_teams = {}                # there's no two word's team this time
two_word_cities = ["New England Patriots", "New York Jets", "Kansas City Chiefs",
                   "Los Angeles Chargers", "New York Giants", "Green Bay Packers",
                   "New Orleans Saints","Tampa Bay Buccaneers", "Los Angeles Rams", 
                   "San Francisco 49ers"]
ct_nm1 = []
nm = []   
for item in nfl["team"]:
    x = item.split()
    nm.append(x[-1])
    if item in two_word_cities:
        ct_nm1.append(x[0:2])
    else:
        ct_nm1.append(x[0])
        
        
ct_nm=[]
for item in ct_nm1:
    if type(item)==list:
        l = " ".join(str(g) for g in item)
        ct_nm.append(l)
    else:
        ct_nm.append(item)


nfl["team"] = ct_nm
nfl["names"] = nm
    
    
nfl

#------------- STR TO FLOAT ENTRIES ----------------

nfl["W"] = nfl["W"].astype('float')
nfl["L"] = nfl["L"].astype('float')


CT['Population (2016 est.)[8]'] = CT['Population (2016 est.)[8]'].astype('float')


#-------------- RATIO W/L CALCULATION  -------------
nfl["W/L"] = nfl["W"]/(nfl["W"]+nfl["L"])





CT

'''
New York City    GiantsJets
Los Angeles      RamsChargers
San Francisco Bay Area   49ersRaiders'''




#----------FIXING NEW YORKS PROBLEM--------------------
ny_df =nfl.loc[nfl['team']=="New York"]
Ny = pd.DataFrame([["New York City",2018,  sum(ny_df["W"]), sum(ny_df["L"]),"GiantsJets",
                   np.mean(ny_df["W/L"])]], columns=['team',"year",'W', "L", 'names','W/L'])

nfl = pd.concat([nfl, Ny])



#----------FIXING LOS ANGELES PROBLEM--------------------
la_df = nfl.loc[nfl['team']=="Los Angeles"]
La = pd.DataFrame([["Los Angeles",2018,  sum(la_df["W"]), sum(la_df["L"]),"RamsChargers",
                   np.mean(la_df["W/L"])]], columns=['team',"year","W", "L", 'names','W/L'])
nfl = pd.concat([nfl, La])


#------- FIXING SAN FRAN BAYS PROBLEM ---------------
sa_df = nfl.loc[nfl['team'].isin(["San Francisco", "Oakland"])]


Sa = pd.DataFrame([["San Francisco Bay Area",2018,  sum(sa_df["W"]),
                    sum(sa_df["L"]),"49ersRaiders",
                   np.mean(sa_df["W/L"])]], columns=['team',"year","W", "L", 'names','W/L'])

nfl = pd.concat([nfl, Sa])






#------------ DELETING SAME REGIONS TEAMS  --------------------
del_same_region = ["Giants", "49ers", "Jets", "Raiders", "Rams", "Chargers"]

nfl = nfl[~nfl["names"].isin(del_same_region)]
nfl.sort_values('team', inplace=True)
nfl=nfl.reset_index(drop=True) #FIXING INDEX

#nfl





final_df = pd.merge(CT, nfl, how="inner", left_on="NFL", right_on="names")
final_df = final_df[["Metropolitan area", "Population (2016 est.)[8]", "NFL","W", "L", "W/L"]]
final_df.sort_values('Metropolitan area')
final_df.reset_index(drop=True, inplace=True)






def nfl_correlation(): 
    
    DF = final_df.copy()
    
    population_by_region = DF["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = DF["W/L"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]



nfl_correlation()

